<a href="https://colab.research.google.com/github/AchuAchii/PEA-101/blob/main/testPEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
###ใส่ไฟล์แล้วจะได้กราฟออกมา
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.dates as mdates

plt.style.use("default")

uploaded = files.upload()
filename = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='cp874')

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

df['year']  = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day']   = df['date'].dt.day
df['time']  = df['date'].dt.time
df['t_as_dt'] = pd.to_datetime(df['date'].dt.strftime('%H:%M:%S'))

def plot_month_by_time(month: int, year: int):
    """Draw 1 line per day: time of day vs kW."""
    filt = (df['month'] == month) & (df['year'] == year)
    sub  = df.loc[filt].copy()

    if sub.empty:
        print(f'No data for {month:02d}/{year}')
        return

    pivot = sub.pivot_table(index='t_as_dt',
                            columns='day',
                            values='kwhexp',
                            aggfunc='mean')

    plt.figure(figsize=(15, 6))

    for day in pivot.columns:
        plt.plot(pivot.index, pivot[day], label=f'Day {day:02d}', linewidth=1)

    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)

    plt.title(f'Power Usage (kW) — {month:02d}/{year} (one line per day)')
    plt.xlabel('Time of Day')
    plt.ylabel('kW')
    plt.grid(True, which='both', axis='both', linestyle='--', alpha=0.3)

    plt.legend(title='Day', bbox_to_anchor=(1.02, 1), loc='upper left', ncol=1, fontsize='small')
    plt.tight_layout()
    plt.show()

month_dd = widgets.Dropdown(
    options=[(f'{i:02d}', i) for i in range(1, 13)],
    value=1,
    description='Month:'
)

years = sorted(df['year'].unique())
year_dd = widgets.Dropdown(
    options=years,
    value=years[0],
    description='Year:'
)

out = widgets.Output()

def refresh_plot(change=None):
    with out:
        clear_output(wait=True)
        plot_month_by_time(month_dd.value, year_dd.value)

refresh_plot()
month_dd.observe(refresh_plot, names='value')
year_dd.observe(refresh_plot, names='value')

display(widgets.HBox([month_dd, year_dd]), out)

Saving 439-พลังงานแสงอาทิตย์.csv to 439-พลังงานแสงอาทิตย์ (17).csv


<ipython-input-32-1186751546>:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['t_as_dt'] = pd.to_datetime(df['date'].dt.strftime('%H:%M:%S'))


Output()

In [31]:
#ใส่ไฟล์แล้วจะได้ALARM PE
import io, numpy as np, pandas as pd
from google.colab import files

uploaded = files.upload()
filename  = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]),
                 encoding='cp874')

date_col   = 'date'
energy_col = 'kwhexp'
mw_col     = 'mwตามสัญญา'

df['PE_pct'] = (
    df[energy_col] /
    (df[mw_col] * 1000 * 0.25)
) * 100

df['Alarm PE'] = np.select(
    [df['PE_pct'] > 24,
     df['PE_pct'] < 20],
    ['HIGH', 'LOW'],
    default='MEDIUM'
)
csv_name = 'with_alarm_pe.csv'
df.to_csv(csv_name, index=False, encoding='utf-8-sig')
files.download(csv_name)

def colour(val):
    if val == 'HIGH':   return 'background-color:red;color:white'
    if val == 'MEDIUM': return 'background-color:yellow;color:black'
    if val == 'LOW':    return 'background-color:green;color:white'
    return ''

styled = df.style.applymap(colour, subset=['Alarm PE'])

xlsx_name = 'with_alarm_pe_colored.xlsx'
styled.to_excel(xlsx_name, index=False, engine='openpyxl')
files.download(xlsx_name)

print('✅ ทั้ง CSV และ Excel (มีสี) ถูกดาวน์โหลดเรียบร้อย')

Saving 439-พลังงานแสงอาทิตย์.csv to 439-พลังงานแสงอาทิตย์ (16).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-31-3403650801>:41: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = df.style.applymap(colour, subset=['Alarm PE'])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ ทั้ง CSV และ Excel (มีสี) ถูกดาวน์โหลดเรียบร้อย
